In [1]:
import re
import os
import sys
import random
import pathlib
import time
import numpy as np
import pandas as pd
from collections import Counter
from PIL import Image

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as po

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.linear_model import SGDClassifier
from sklearn import metrics

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2022-01-13 10:11:38.293654: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-13 10:11:38.293686: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# sex = {'girl': '1girl', 'tomboy': '1girl', 'boy': '1boy', 'trap': '1boy'}
# labels = ['girl', 'tomboy', 'boy', 'trap']

# df_raw = dict()
# feature_raw = dict()
# image_raw = dict()

# for label in labels:
#     df_raw[label] = pd.read_csv('./images/' + label + '.csv', header=0, converters={'tags':lambda x: x.split()}).sample(4500)
#     df_raw[label]['Rating'] = df_raw[label]['tags'].apply(lambda x: x[-4][7:])
#     df_raw[label]['tags'] = df_raw[label]['tags'].apply(lambda x: x[x.index(sex[label]):-4])
#     df_raw[label]['tags'] = df_raw[label]['tags'].apply(lambda x: [e for e in x if e != sex[label] and e != label])
#     # df_raw[label]['flag'] = df_raw[label]['tags'].apply(lambda x: ('female_focus' in x) or ('male_focus' in x))
#     counter = Counter()
#     for row in df_raw[label]['tags']:
#         counter.update(row)
#     feature_raw[label] = np.array([[k, v] for k, v in zip(counter.keys(), counter.values())], dtype=object)
#     feature_raw[label] = feature_raw[label][np.lexsort((feature_raw[label][:, 0], feature_raw[label][:, 1]))]

In [3]:
# X = []
# y = []

# for label in labels:
#     for ID in df_raw[label]['id'].values:
#         img = np.array(tf.keras.utils.load_img('./images/' + label + '/' + str(ID) + '.jpg', color_mode='grayscale'))
#         X.append(img)
#         y.append(label)

# X = np.array(X)
# X = X.reshape(4500*4, 128, 128, 1)
# y = np.array(y)
# indices = np.arange(X.shape[0])
# np.random.shuffle(indices)
# X, y = X[indices], y[indices]

In [4]:
# class CNN:
#     def __init__(self, labels):
#         self.label_encoder = tf.keras.layers.StringLookup(
#             output_mode='one_hot',
#             vocabulary=labels,
#         )
#         self.test_model = tf.keras.Sequential([
#             tf.keras.layers.Rescaling(1./255),
#             tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(128,128)),
#             tf.keras.layers.MaxPooling2D(),
#             tf.keras.layers.Conv2D(32, 3, activation='relu'),
#             tf.keras.layers.MaxPooling2D(),
#             tf.keras.layers.Conv2D(32, 3, activation='relu'),
#             tf.keras.layers.MaxPooling2D(),
#             tf.keras.layers.Flatten(),
#             tf.keras.layers.Dense(128, activation='relu'),
#             tf.keras.layers.Dense(self.label_encoder.vocabulary_size())
#         ])

#     def fit(self, X, y):
#         self.model = tf.keras.models.clone_model(self.test_model)
#         self.model.compile(
#             loss="categorical_crossentropy", optimizer="adam", metrics=["categorical_accuracy"]
#         )
#         self.model.fit(X, self.label_encoder(y).numpy(), batch_size=128, epochs=10, validation_split=0.2)
#     def predict(self, X):
#         y = self.model.predict(X)
#         return np.array([self.label_encoder.get_vocabulary()[x] for x in np.argmax(y, axis=1)])

In [5]:
# model = CNN()
# model.fit(X, y)

In [6]:
# def trans_bg(arr):
#     temp = arr.copy()
#     temp[arr == 'tomboy'] = 'girl'
#     temp[arr == 'girl'] = 'girl'
#     temp[arr == 'trap'] = 'boy'
#     temp[arr == 'boy'] = 'boy'
#     return temp
# y_bg = trans_bg(y)

In [7]:
# model = CNN(['girl', 'boy'])
# model.fit(X, y_bg)

In [8]:
# def trans_bgish(arr):
#     temp = arr.copy()
#     temp[arr == 'tomboy'] = 'girlish'
#     temp[arr == 'girl'] = 'boyish'
#     temp[arr == 'trap'] = 'girlish'
#     temp[arr == 'boy'] = 'boyish'
#     return temp
# y_bgish = trans_bgish(y)

In [9]:
# model = CNN(['girlish', 'boyish'])
# model.fit(X, y_bgish)

In [10]:
data_dir = './Dataset_bg'
batch_size = 32
img_height = 128
img_width = 128


train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    color_mode='grayscale',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    color_mode='grayscale',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 18000 files belonging to 2 classes.
Using 14400 files for training.


2022-01-13 10:11:40.923511: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-13 10:11:40.923535: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-13 10:11:40.923552: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (archlinux): /proc/driver/nvidia/version does not exist
2022-01-13 10:11:40.923766: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 18000 files belonging to 2 classes.
Using 3600 files for validation.


In [11]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:
num_classes = 2

model = tf.keras.Sequential([ 
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [13]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

Epoch 1/10
450/450 [==============================] - 129s 285ms/step - loss: 0.6906 - accuracy: 0.5304 - val_loss: 0.6871 - val_accuracy: 0.5367
Epoch 2/10
450/450 [==============================] - 130s 289ms/step - loss: 0.6753 - accuracy: 0.5755 - val_loss: 0.6807 - val_accuracy: 0.5683
Epoch 3/10
450/450 [==============================] - 128s 284ms/step - loss: 0.6633 - accuracy: 0.5980 - val_loss: 0.6931 - val_accuracy: 0.5628
Epoch 4/10
450/450 [==============================] - 128s 284ms/step - loss: 0.6449 - accuracy: 0.6238 - val_loss: 0.7036 - val_accuracy: 0.5564
Epoch 5/10
450/450 [==============================] - 128s 284ms/step - loss: 0.5969 - accuracy: 0.6728 - val_loss: 0.7480 - val_accuracy: 0.5481
Epoch 6/10
450/450 [==============================] - 128s 284ms/step - loss: 0.5173 - accuracy: 0.7314 - val_loss: 0.8252 - val_accuracy: 0.5489
Epoch 7/10
450/450 [==============================] - 127s 283ms/step - loss: 0.4370 - accuracy: 0.7835 - val_loss: 1.0702 -

In [14]:
data_dir = './Dataset_bgish'
batch_size = 32
img_height = 128
img_width = 128


train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    color_mode='grayscale',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    color_mode='grayscale',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 18000 files belonging to 2 classes.
Using 14400 files for training.
Found 18000 files belonging to 2 classes.
Using 3600 files for validation.


In [15]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [16]:
num_classes = 2

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [17]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

Epoch 1/10
450/450 [==============================] - 131s 291ms/step - loss: 0.6798 - accuracy: 0.5664 - val_loss: 0.6639 - val_accuracy: 0.6081
Epoch 2/10
450/450 [==============================] - 130s 288ms/step - loss: 0.6634 - accuracy: 0.6062 - val_loss: 0.6550 - val_accuracy: 0.6289
Epoch 3/10
450/450 [==============================] - 130s 289ms/step - loss: 0.6526 - accuracy: 0.6196 - val_loss: 0.6546 - val_accuracy: 0.6344
Epoch 4/10
450/450 [==============================] - 130s 288ms/step - loss: 0.6384 - accuracy: 0.6381 - val_loss: 0.6661 - val_accuracy: 0.6186
Epoch 5/10
450/450 [==============================] - 131s 292ms/step - loss: 0.6154 - accuracy: 0.6639 - val_loss: 0.7161 - val_accuracy: 0.6061
Epoch 6/10
450/450 [==============================] - 131s 291ms/step - loss: 0.5677 - accuracy: 0.7037 - val_loss: 0.7642 - val_accuracy: 0.6053
Epoch 7/10
450/450 [==============================] - 128s 283ms/step - loss: 0.4953 - accuracy: 0.7522 - val_loss: 0.8601 -

In [18]:
str1 = '''
Epoch 3/3
450/450 [==============================] - 128s 284ms/step - loss: 0.6633 - accuracy: 0.5980 - val_loss: 0.6931 - val_accuracy: 0.5628
'''
str2 = '''
Epoch 3/3
450/450 [==============================] - 130s 289ms/step - loss: 0.6526 - accuracy: 0.6196 - val_loss: 0.6546 - val_accuracy: 0.6344
'''

print('boy/girl:\n', str1, '\nboyish\girlish:\n', str2)

boy/girl:
 
Epoch 3/3
450/450 [==============================] - 128s 284ms/step - loss: 0.6633 - accuracy: 0.5980 - val_loss: 0.6931 - val_accuracy: 0.5628
 
boyish\girlish:
 
Epoch 3/3
450/450 [==============================] - 130s 289ms/step - loss: 0.6526 - accuracy: 0.6196 - val_loss: 0.6546 - val_accuracy: 0.6344

